In [1]:
import pandas as pd
import numpy as np
from data.data_dist import StreamDataGeneratorAspenDIST
from system.aspenplusconnector import ASPENConnector
import matplotlib.pyplot as plt

In [ ]:
system = ASPENConnector("C:\\Users\\zrasmussen\\dataset\\multi-comp distillation.bkp")
system.connect()

In [3]:

FLOW_A_path = "\\Data\\Streams\\FEED\\Input\\FLOW\\MIXED\\N-BUTANE"
FLOW_B_path = "\\Data\\Streams\\FEED\\Input\\FLOW\\MIXED\\N-PENTAN"
FLOW_C_path = "\\Data\\Streams\\FEED\\Input\\FLOW\\MIXED\\N-HEXANE"
num_tray_path = "\\Data\\Blocks\\B1\\Input\\NSTAGE"
feed_tray_path = "\\Data\\Blocks\\B1\\Input\\FEED_STAGE\\FEED"
dist_rate_path = "\\Data\\Blocks\\B1\\Input\\BASIS_D"
reflux_ratio_path = "\\Data\\Blocks\\B1\\Input\\BASIS_RR"
pressure_block_path = "\\Data\\Blocks\\B1\\Input\\PRES1"
pressure_flow_path = "\\Data\\Streams\\FEED\\Input\\PRES\\MIXED"

stream_list = [
        "FEED",
        "DIST",
        "BOTT",
]
all_in_streams = [
    "FEED",
]
all_out_streams = [
    "DIST",
    "BOTT",
]
all_components = [
    "N-BUTANE",
    "N-PENTAN",
    "N-HEXANE",
]
all_blocks = [
    "B1",
]


def Simulate(num,df): #num = which column 1 or 2
    system.aspen.Tree.FindNode(FLOW_A_path).Value = df['FEED_BUT'][num-1]
    system.aspen.Tree.FindNode(FLOW_B_path).Value = df['FEED_PEN'][num-1]
    system.aspen.Tree.FindNode(FLOW_C_path).Value = df['FEED_HEX'][num-1]
    system.aspen.Tree.FindNode(num_tray_path).Value = df['#-TRAYS'][num-1]
    system.aspen.Tree.FindNode(feed_tray_path).Value = df['FEED_TRAY'][num-1]
    system.aspen.Tree.FindNode(dist_rate_path).Value = df['DIST_RATE'][num-1] + 1e-5
    system.aspen.Tree.FindNode(reflux_ratio_path).Value = df['REF_RAT'][num-1]
    system.aspen.Tree.FindNode(pressure_block_path).Value = df['PRES'][num-1]
    system.aspen.Tree.FindNode(pressure_flow_path).Value = df['PRES'][num-1]
    system.aspen.Reinit()
    system.aspen.Engine.Run2()
    data_generator = StreamDataGeneratorAspenDIST(system=system.aspen)
    data_dict = data_generator.generate(all_streams_list=stream_list,
                                        all_in_streams_list=all_in_streams,
                                        all_out_streams_list=all_out_streams,
                                        all_components=all_components,
                                        all_blocks = all_blocks)
    
    return data_dict

def error(data_dict,num,df):
    op_cost = data_dict['B1']['Operating Cost Condenser'] + data_dict['B1']['Operating Cost Reboiler']
    cap_cost = data_dict['B1']['Investment Cost']
    dist_a = data_dict['DIST']['MOLEFLOW']['N-BUTANE']
    dist_b = data_dict['DIST']['MOLEFLOW']['N-PENTAN']
    dist_c = data_dict['DIST']['MOLEFLOW']['N-HEXANE']
    print('operating cost')
    print(df['OP_COST'][num-1])
    print('capital cost')
    print(df['CAP_COST'][num-1])
    print(df['DIST_BUT'][num-1])
    print(df['DIST_PEN'][num-1])
    print(df['DIST_HEX'][num-1])
    op_cost_err = np.abs(op_cost - df['OP_COST'][num-1]) / op_cost
    cap_cost_err = np.abs(cap_cost - df['CAP_COST'][num-1]) / cap_cost
    dist_a_err = np.abs(dist_a - df['DIST_BUT'][num-1]) / dist_a
    dist_b_err = np.abs(dist_b - df['DIST_PEN'][num-1]) / dist_b
    dist_c_err = np.abs(dist_c - df['DIST_HEX'][num-1]) / dist_c

    df_err = pd.DataFrame([[op_cost_err,cap_cost_err,dist_a_err,dist_b_err,dist_c_err,op_cost,df['OP_COST'][num-1],cap_cost,df['CAP_COST'][num-1],
                           dist_a,df['DIST_BUT'][num-1],dist_b,df['DIST_PEN'][num-1],dist_c,df['DIST_HEX'][num-1]]],
                            columns = ['Operating Cost Error', 'Capital Cost Error', 'Dist A Error',
                                'Dist B Error','Dist C Error','True Operating Cost','Pred Operating Cost','True Capital Cost','Pred Capital Cost',
                                      'True Dist A','Pred Dist A','True Dist B','Pred Dist B','True Dist C','Pred Dist C'])
    return df_err

def create_report():
    df_err = pd.DataFrame(columns = ['Operating Cost Error', 'Capital Cost Error', 'Dist A Error',
                                'Dist B Error','Dist C Error','True Operating Cost','Pred Operating Cost','True Capital Cost','Pred Capital Cost',
                                                'True Dist A','Pred Dist A','True Dist B','Pred Dist B','True Dist C','Pred Dist C','TAG'])
    for a in np.linspace(165,329,3):
        for b in np.linspace(165,329,3):
            for c in np.linspace(1,165,3):
                
                try:
                    filename = 'nn_relu_val_values/RELU{}-{}-{} nn  validation values'.format(a,b,c)
                    df = pd.read_csv(filename)
                    for num in range(1,3):
                        if df['FEED_BUT'][num-1] != 0:
                            data_dict = Simulate(num,df)
                            df_err_temp = error(data_dict, num,df)
                            df_err_temp['TAG'] = "{}-{}-{} col {}".format(a,b,c,num)
                            df_err = pd.concat([df_err,df_err_temp])
                except:
                    print('no file {}'.format(filename))
    return df_err

def manually_simulate(flow_A,flow_B,flow_C,num_tray,feed_tray,dist_rate,ref_rate,pressure):
    system.aspen.Tree.FindNode(FLOW_A_path).Value = flow_A
    system.aspen.Tree.FindNode(FLOW_B_path).Value = flow_B
    system.aspen.Tree.FindNode(FLOW_C_path).Value = flow_C
    system.aspen.Tree.FindNode(num_tray_path).Value = num_tray
    system.aspen.Tree.FindNode(feed_tray_path).Value = feed_tray
    system.aspen.Tree.FindNode(dist_rate_path).Value = dist_rate
    system.aspen.Tree.FindNode(reflux_ratio_path).Value = ref_rate/dist_rate
    system.aspen.Tree.FindNode(pressure_block_path).Value = pressure
    system.aspen.Tree.FindNode(pressure_flow_path).Value = pressure
    system.aspen.Reinit()
    system.aspen.Engine.Run2()
    data_generator = StreamDataGeneratorAspenDIST(system=system.aspen)
    data_dict = data_generator.generate(all_streams_list=stream_list,
                                        all_in_streams_list=all_in_streams,
                                        all_out_streams_list=all_out_streams,
                                        all_components=all_components,
                                        all_blocks = all_blocks)
    
    return data_dict

In [4]:
"""num=1
data_dict = Simulate(num)
df_err = error(data_dict,num)
print(df_err)"""

#df_err = create_report()
flow_A = 310.477
flow_B = 310.477
flow_C = 319.885
num_tray = 5
feed_tray = 3
dist_rate = 492.619
ref_rate = 228.416
pressure = 1.9
data_dict = manually_simulate(flow_A=flow_A,flow_B=flow_B,flow_C=flow_C,num_tray=num_tray,feed_tray=feed_tray,dist_rate=dist_rate,ref_rate=ref_rate
                              ,pressure=pressure)

retrieving Stream: FEED, Component: N-BUTANE
retrieving Stream: FEED, Component: N-PENTAN
retrieving Stream: FEED, Component: N-HEXANE
retrieving Stream: DIST, Component: N-BUTANE
retrieving Stream: DIST, Component: N-PENTAN
retrieving Stream: DIST, Component: N-HEXANE
retrieving Stream: BOTT, Component: N-BUTANE
retrieving Stream: BOTT, Component: N-PENTAN
retrieving Stream: BOTT, Component: N-HEXANE
{'FEED': {'MOLEFLOW': {'N-BUTANE': 310.477, 'N-PENTAN': 310.477, 'N-HEXANE': 319.885}}, 'DIST': {'MOLEFLOW': {'N-BUTANE': 295.675692, 'N-PENTAN': 167.433272, 'N-HEXANE': 29.5100355}}, 'BOTT': {'MOLEFLOW': {'N-BUTANE': 14.801308, 'N-PENTAN': 143.043728, 'N-HEXANE': 290.374964}}, 'B1': {'COND-DUTY': -1192075.31, 'REB-DUTY': 1490888.99, '#-TRAYS': 5, 'FEED_TRAY': 3, 'DIST_RATE': 492.619, 'REFLUX_RATIO': 0.46367679688, 'Investment Cost': 1399043.508470006, 'Operating Cost Reboiler': 1936260.050663107, 'Operating Cost Condenser': 205842.70140299937, 'PRES': 2.0, 'TOP_TEMP': 29.5156537, 'REB_TE